# Library Installation

In [ ]:
!pip install numpy
!pip install nltk
!pip install keras tensorflow
!pip install Flask Flask-ngrok
!pip install chatterbot chatterbot-corpus
!pip install -U scikit-fuzzy
!pip install matplotlib

# Weather Data 📈

In [ ]:
%run weather_scrap.ipynb

In [ ]:
%run fuzzy.ipynb

# Training Model Section

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

# Constructor
lemmatizer = WordNetLemmatizer()

# Load & Read intents Json file
intents = json.loads(open('intents.json').read())

# Entries List for word Part ---
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']  # ignore those letter

# Tokenisation
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # Take Each Word and Tokenize it
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))

        # Check the classes already in the document
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatizer
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Training Data Part ---
training = []
output_empty = [0] * len(classes)

# Generate Bag of Words
for document in documents:
    bag = []
    # List of Tokenized words for the pattern
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    # Create Bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # Output From List
    output_row = list(output_empty)
    # Save the index 1 from the document into output
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Shuffle the Training Data and Put into np.array
random.shuffle(training)
training = np.array(training)

# Create Train and Test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# Neural Network
# Create Training Model Part ---
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and Saving the model
hist = model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model Created")

# Chatbot Section 💻



In [ ]:
import random
import json
import pickle
import numpy as np

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

import re

# Constructor
lemmatizer = WordNetLemmatizer()

# Load & Read intents Json file
intents = json.loads(open('intents.json').read())
weather_intents = json.loads(open('weather_intents.json').read())

# Load the Pickle file & Model file
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

app = Flask(__name__)
run_with_ngrok(app)

month_list = [
"january",
"february",
"march",
"april",
"may",
"june",
"july",
"august",
"september",
"october",
"november",
"december"
]

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get", methods=["POST"])
def chatbot_response():
    # Get message from user input
    msg = request.form["msg"]
    # Convert the sentence to lower case
    lower_msg = msg.lower()
    # Remove special characters
    filter_msg = re.sub('[^A-Za-z ]+', '', lower_msg)
    filter_msg2 = re.sub('[^A-Za-z ]+', '', msg)
    # Spilt words in a sentence
    spilt_msg = [word for word in filter_msg.split()]

    # Check boolean state for matching results in weather_intents and month_list (destination, months)
    found = False

    for msg_word in spilt_msg:
        intents_list = weather_intents['intent_dict']

        # Loop through weather_intents
        for i in intents_list:
            # Check destination (cities name) and convert to lower case
            check = i['destination'].lower()
            # Spilt cities name
            spilt_check = [word for word in check.split()]

            # Assign empty string for the variable
            final_destination_name = ""

            if msg_word in spilt_check:
                # Get current index of the word in the string array
                current_index = spilt_msg.index(msg_word)

                if msg_word == i['destination'].lower():
                    # For city name that has only 1 word
                    destination_name = str(spilt_msg[current_index])
                else:
                    # For city name with more than 1 word
                    destination_name = str(spilt_msg[current_index - 1]) + " " + str(spilt_msg[current_index])

                # Assign the combined words for a city name to final_destination_name
                final_destination_name = destination_name

            # If the city name matched with the value in the intents
            if final_destination_name == i['destination'].lower():
                lower_destination_str = i['destination'].lower()

                for msg_word in spilt_msg:
                    #If month is found in the month_list
                    if msg_word in month_list:
                        # Used to check if message from user contains both city's name and month
                        found = True

                        # Information to be displayed
                        prob = i['months'][msg_word]['probability']
                        avg_temp_f = i['months'][msg_word]['avg_temp']
                        avg_temp_c = (avg_temp_f - 32) * 5/9
                        avg_temp_c = "{:.2f}".format(avg_temp_c)
                        clearer = i['months'][msg_word]['clearer']
                        precipitation = i['months'][msg_word]['precipitation']
                        rainfall = i['months'][msg_word]['rainfall']
                        windspeed = i['months'][msg_word]['windspeed']

                        # If probability more than 50
                        if(prob > 50):
                            res = "<b><u>" + "Average Weather information:" + "</u></b>" + "<br>" + "Temperature: " + str(avg_temp_c) + "°C" + "<br>" + "Clear days: " + str(clearer) + "%" + "<br>" + "Precipitation: " + str(precipitation) + "d" + "<br>" + "Rainfall: " + str(rainfall) + "″" + "<br>" + "Windspeed (mph): " + str(windspeed) + "<br>" + "<br>" + "Good option! " + lower_destination_str.title() + " is suitable to go in " + msg_word.capitalize() + ".👍"
                        else:
                            # Find months that has probability more than 50
                            destination_months = i['months']
                            all_keys = []
                            for key, value in destination_months.items():
                                if(value['probability'] > 50):
                                    all_keys.append(key.capitalize())

                            # Output message part 1 (All weather information for that particular city on particular month)
                            res1 = "<b><u>" + "Average weather information:" + "</u></b>" + "<br>" + "Temperature: " + str(avg_temp_c) + "°C" + "<br>" + "Clear days: " + str(clearer) + "%" + "<br>" + "Precipitation: " + str(precipitation) + "d" + "<br>" + "Rainfall: " + str(rainfall) + "″" + "<br>" + "Windspeed (mph): " + str(windspeed) + "<br>" + "<br>" + "Hmm... I would not recommend you to go " + lower_destination_str.title() + " in " + msg_word.capitalize() + ".😫"
                            # Output message part 2 (For suitable months to visit)
                            res2 = ', '.join(all_keys)
                            res = res1 + " <br><br> Perhaps, you may choose to visit " + lower_destination_str.title() + " in " + res2 + "."

    # When both (city name AND month) is not found, get response through intent.json
    if found == False:
        if filter_msg2.startswith("my name is"):
            name = filter_msg2[11:]
            ints = predict_class(filter_msg2, model)
            res1 = get_response(ints, intents)
            res = res1.replace("{n}", name)
        elif filter_msg2.startswith("hi my name is"):
            name = filter_msg2[14:]
            ints = predict_class(filter_msg2, model)
            res1 = get_response(ints, intents)
            res = res1.replace("{n}", name)
        elif filter_msg2.startswith("i am"):
            name = filter_msg2[5:]
            ints = predict_class(filter_msg2, model)
            res1 = get_response(ints, intents)
            res = res1.replace("{n}", name)
        else:
            ints = predict_class(msg, model)
            res = get_response(ints, intents)

    return res

# Function : Clearing sentences
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function : Bag
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
            if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

# Function : Predict the sentences
def predict_class(sentence, model):
    #bow = bag_of_words(sentence)
    bow = bag_of_words(sentence, words, show_details=False)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25      # 25%
    results =[[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

# Function : Response
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

if __name__ == "__main__":
    app.run()
